In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders as ce
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

In [3]:
df_emp = pd.read_csv("../input/collector/employees.csv")

df_comment_train = pd.read_csv("../input/collector/train_comments.csv")
df_issues_train = pd.read_csv("../input/collector/train_issues.csv")
sample_solution = pd.read_csv("../input/collector/sample_solution.csv")

df_comment_test = pd.read_csv("../input/collector/test_comments.csv")
df_issues_test = pd.read_csv("../input/collector/test_issues.csv")

In [4]:
df_issues_train = df_issues_train[df_issues_train['overall_worklogs'] < 500000]
df_issues_train = df_issues_train[['id', 'project_id', 'assignee_id', 'creator_id', 'overall_worklogs', 'created', 'summary']]

In [5]:

encoder = ce.OrdinalEncoder(cols=['position','hiring_type','payment_type','salary_calculation_type','english_level'], return_df=True , verbose = None)
df_emp = encoder.fit_transform(df_emp)
df_emp = df_emp.rename(columns={"id": "assignee_id"})
df_emp = df_emp.drop(['full_name'], axis =1)

In [7]:
# статистики времени выполнения задач по исполнителям мерджить по creator_id
df_temp_3 = df_issues_train.pivot_table(index = 'creator_id', 
                            values=['overall_worklogs'],
                            aggfunc=[ 'count', 'mean', 'sum', 'max', 'min']).fillna(0).astype('int64')
df_temp_3.columns = [f'{str(i[0])}_times_by_creators' for i in df_temp_3.columns]
df_temp_3 = df_temp_3.reset_index()

In [8]:
# статистики времени выполнения задач по исполнителям мерджить по assignee_id
df_temp_2 = df_issues_train.pivot_table(index = 'assignee_id', 
                            values=['overall_worklogs'],
                            aggfunc=[ 'count', 'mean', 'sum', 'max', 'min']).fillna(0).astype('int64')
df_temp_2.columns = [f'{str(i[0])}_times_by_assignee' for i in df_temp_2.columns]
df_temp_2 = df_temp_2.reset_index()

In [9]:
# статистики времени выполнения задач по проектам мерджить по project_id
df_temp_1 = df_issues_train.pivot_table(index = 'project_id', 
                            values=['overall_worklogs'],
                            aggfunc=[  'count', 'mean', 'sum', 'max', 'min']).fillna(0).astype('int64')
df_temp_1.columns = [f'{str(i[0])}_times_by_projects' for i in df_temp_1.columns]
df_temp_1 = df_temp_1.reset_index()

In [10]:
train = df_issues_train.merge(df_temp_1, on='project_id',how='left'
                             ).merge(df_temp_2, on='assignee_id',how='left'
                                    ).merge(df_temp_3, on='creator_id',how='left'
                                           ).merge(df_emp, on='assignee_id',how='left'
                                                  )

In [11]:
df_temp_6 = train.pivot_table(index = 'position', 
                            values=['overall_worklogs'],
                            aggfunc=[ 'count', 'max', 'mean', 'sum', 'min']).fillna(0).astype('int64')
df_temp_6.columns = [f'{str(i[0])}_times_by_position' for i in df_temp_6.columns]
df_temp_6 = df_temp_6.reset_index()
train = train.merge(df_temp_6, on='position',how='left')

In [14]:
train.created = pd.to_datetime(train.created, errors='coerce')
def prep1(df):
    df['dayofweek'] = df.created.dt.dayofweek
    df['hour'] = df.created.dt.hour
    df['dayofyear'] = df.created.dt.dayofyear
    df['week'] = df.created.dt.isocalendar().week.astype(int)
    df['day'] = df.created.dt.day
    df['month'] = df.created.dt.month
    df['year'] = df.created.dt.year
    df = df.drop(['created'], axis=1)
    return df
train = prep1(train).reset_index()

In [15]:
df_comment_train = pd.read_csv("../input/collector/train_comments.csv")

df_comment_train['len_comment'] = df_comment_train['text'].apply(lambda x: len(x.split(' ')))
agg_stst_by_issues = df_comment_train.pivot_table(index = 'issue_id', 
                            values=['len_comment'],
                            aggfunc=[  'count', 'mean', 'sum', 'max', 'min']).fillna(0)
agg_stst_by_issues.columns = [f'{str(i[0])}_len_comments_by_issues' for i in agg_stst_by_issues.columns]
agg_stst_by_issues = agg_stst_by_issues.reset_index()
agg_stst_by_issues = agg_stst_by_issues.rename(columns={"issue_id": "id"})
train = train.merge(agg_stst_by_issues, on='id', how='left')

In [16]:
# статистики времени выполнения задач по месяцам мерджить по month
df_temp_4 = train.pivot_table(index = 'month', 
                            values=['overall_worklogs'],
                            aggfunc=[  'mean', 'sum', 'min', 'max', 'count']).fillna(0).astype('int64')
df_temp_4.columns = [f'{str(i[0])}_times_by_month' for i in df_temp_4.columns]
df_temp_4 = df_temp_4.reset_index()

In [17]:
# статистики времени выполнения задач по неделям в году мерджить по month
df_temp_5 = train.pivot_table(index = 'week', 
                            values=['overall_worklogs'],
                            aggfunc=[  'mean', 'sum', 'min', 'max', 'count']).fillna(0).astype('int64')
df_temp_5.columns = [f'{str(i[0])}_times_by_week' for i in df_temp_5.columns]
df_temp_5 = df_temp_5.reset_index()
train = train.merge(df_temp_4, on='month',how='left').merge(df_temp_5, on='week',how='left')

In [19]:
df_comment_train['len_comment'] = df_comment_train['text'].apply(lambda x: len(x.split(' ')))
df_comment_train = df_comment_train.pivot_table(index = 'author_id', 
                            values=['len_comment'],
                            aggfunc=[  'count', 'mean', 'sum', 'max', 'min']).fillna(0)
df_comment_train.columns = [f'{str(i[0])}_comments_by_authors' for i in df_comment_train.columns]
df_comment_train = df_comment_train.reset_index()

In [20]:
df_comment_train_assignee_id =  df_comment_train.rename(columns={"author_id": "assignee_id"})
df_comment_train_creator_id =  df_comment_train.rename(columns={"author_id": "creator_id"})
train = train.merge(df_comment_train_assignee_id, on="assignee_id", how='left')#.merge(df_comment_train_creator_id, on="creator_id", how='left')
train = train.fillna(0)

In [22]:
X = train.drop(['overall_worklogs', 'index', 'summary'], axis =1)
y = train[['overall_worklogs']]

# **Текст**

In [23]:
transformers_text_features = pd.read_csv('../input/collector/transformers_text_features.csv')
transformers_text_features = transformers_text_features.drop(['summary_total_sum_comm', 'created', 'key', 'summary', 'project_id', 'assignee_id', 'creator_id', 'overall_worklogs', 'total_sum_comments_by_issues'], axis=1)

In [24]:
X = X.merge(transformers_text_features, on='id', how='left')

In [26]:
param1 = {'nan_mode': 'Min', 
         'iterations': 4000, 
         'task_type': "GPU",
         'devices': '-1',
         'eval_metric': 'R2:hints=skip_train~false',
         'gpu_cat_features_storage': 'GpuRam', 
         'leaf_estimation_method': 'Gradient', 
         'grow_policy': 'Depthwise', 
         'boosting_type': 'Plain', 
         'feature_border_type': 'GreedyLogSum', 
         'l2_leaf_reg': 25, 
         'random_strength': 1, 
         'rsm': 1, 
         'boost_from_average': False, 
         'model_size_reg': 0.5, 
         'random_seed': 9, 
         'depth': 7, 
         'border_count': 128, 
         'custom_metric': 'R2', 
         'leaf_estimation_backtracking': 'AnyImprovement', 
         'best_model_min_trees': 1, 
         'min_data_in_leaf': 1, 
         'learning_rate': 0.001, 
         'score_function': 'Cosine', 
         'leaf_estimation_iterations': 5, 
         'bootstrap_type': 'Bayesian', 
         'bagging_temperature': 1,
         'max_leaves': 128,
         'verbose': 500,
        }

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
%%time
clf = CatBoostRegressor(verbose=500).set_params(**param1)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
score = r2_score(y_test, pred)
score

Default metric period is 5 because R2 is/are not implemented for GPU
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric R2 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: -0.2714892	total: 71.1ms	remaining: 4m 44s
500:	learn: 0.1094913	total: 22.6s	remaining: 2m 37s
1000:	learn: 0.3209525	total: 44.5s	remaining: 2m 13s
1500:	learn: 0.4490764	total: 1m 4s	remaining: 1m 48s
2000:	learn: 0.5326612	total: 1m 26s	remaining: 1m 25s
2500:	learn: 0.5909732	total: 1m 48s	remaining: 1m 5s
3000:	learn: 0.6336335	total: 2m 9s	remaining: 43s
3500:	learn: 0.6674768	total: 2m 29s	remaining: 21.4s
3999:	learn: 0.6940697	total: 2m 52s	remaining: 0us
CPU times: user 4min 59s, sys: 1min 35s, total: 6min 34s
Wall time: 4min 46s


0.20107576360343682

In [29]:
# 0.20107576360343682 = 0.165554 on LB

In [30]:
df_emp = pd.read_csv("../input/collector/employees.csv")

df_comment_train = pd.read_csv("../input/collector/train_comments.csv")
df_issues_train = pd.read_csv("../input/collector/train_issues.csv")
sample_solution = pd.read_csv("../input/collector/sample_solution.csv")

df_comment_test = pd.read_csv("../input/collector/test_comments.csv")
df_issues_test = pd.read_csv("../input/collector/test_issues.csv")

In [31]:
df_emp = encoder.transform(df_emp)
df_emp = df_emp.rename(columns={"id": "assignee_id"})
df_emp = df_emp.drop(['full_name'], axis =1)
df_issues_test = df_issues_test[['id', 'project_id', 'assignee_id', 'creator_id',  'created', 'summary']]

In [33]:
test = df_issues_test.merge(df_temp_1, on='project_id',how='left'
                           ).merge(df_temp_2, on='assignee_id',how='left'
                                    ).merge(df_temp_3, on='creator_id',how='left'
                                           ).merge(df_emp, on='assignee_id',how='left')

In [34]:
test = test.merge(df_temp_6, on='position',how='left')
test.created = pd.to_datetime(test.created, errors='coerce')
test = prep1(test).reset_index()

In [36]:
df_comment_train = pd.read_csv("../input/collector/train_comments.csv")

df_comment_train['len_comment'] = df_comment_train['text'].apply(lambda x: len(x.split(' ')))
agg_stst_by_issues = df_comment_train.pivot_table(index = 'issue_id', 
                            values=['len_comment'],
                            aggfunc=[  'count', 'mean', 'sum', 'max', 'min']).fillna(0)
agg_stst_by_issues.columns = [f'{str(i[0])}_len_comments_by_issues' for i in agg_stst_by_issues.columns]
agg_stst_by_issues = agg_stst_by_issues.reset_index()
agg_stst_by_issues = agg_stst_by_issues.rename(columns={"issue_id": "id"})
test = test.merge(agg_stst_by_issues, on='id', how='left')
test = test.merge(df_temp_4, on='month',how='left').merge(df_temp_5, on='week',how='left')

In [38]:
df_comment_test['len_comment'] = df_comment_test['text'].apply(lambda x: len(x.split(' ')))
df_comment_test = df_comment_test.pivot_table(index = 'author_id', 
                            values=['len_comment'],
                            aggfunc=[ 'count', 'mean', 'sum', 'max', 'min']).fillna(0)
df_comment_test.columns = [f'{str(i[0])}_comments_by_authors' for i in df_comment_test.columns]
df_comment_test = df_comment_test.reset_index()

In [39]:
df_comment_test_assignee_id =  df_comment_test.rename(columns={"author_id": "assignee_id"})
df_comment_test_creator_id =  df_comment_test.rename(columns={"author_id": "creator_id"})
test = test.merge(df_comment_test_assignee_id, on="assignee_id", how='left')
test = test.fillna(0)
test_df = test.drop([ 'index', 'summary'], axis =1)

In [41]:
test_transformers_text_features = pd.read_csv('../input/collector/test_transformers_text_features.csv')
test_transformers_text_features = test_transformers_text_features.drop(['summary_total_sum_comm', 'created', 'key', 'summary', 'project_id', 'assignee_id', 'creator_id', 'total_sum_comments_by_issues'], axis=1)
test_df = test_df.merge(test_transformers_text_features, on='id', how='left')

In [43]:
pred = clf.predict(test_df)
pred = pred.astype('int64')
sample_solution.overall_worklogs = pred
sample_solution.to_csv('sub.csv', index=False)